# Data Preparation

Prepare train/val/test splits for coconut tree detection.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-object-detection-on-aerial-imagery/blob/master/notebooks/experiment.ipynb)

In [9]:
# !pip install dl4cv_oda

In [10]:
import requests
import geopandas as gpd
from pathlib import Path
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)

In [11]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded OSM data")

if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Cleaned {count} trees")

if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Downloaded tiles")

In [12]:
if not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"Clipped {stats['total_trees']} trees to {stats['processed']} tiles")
    
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Converted to YOLO format")
    
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42)
    print(f"Split: train={train_count}, val={val_count}, test={test_count}")
    
    config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
    print(f"Config: {config_file}")

print("Data preparation complete")

Data preparation complete


# Model Training

In [13]:
import json
from datetime import datetime
import pandas as pd
from ultralytics import YOLO, RTDETR
import torch

## Configuration

In [14]:
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

SEED = 42
IMG_SIZE = 256
EPOCHS = 200
PATIENCE = 50
BATCH = 16

TUNE_ITERATIONS = 10

MODELS = [
    {"name": "yolov8l", "weights": "yolov8l.pt"},
    {"name": "yolo12l", "weights": "yolo12l.pt"},
    {"name": "rtdetr-l", "weights": "rtdetr-l.pt"},
]

torch.manual_seed(SEED)
exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Experiment: {exp_id}")
print(f"Models: {[m['name'] for m in MODELS]}")

Experiment: 20260111_151930
Models: ['yolov8l', 'yolo12l', 'rtdetr-l']


## Hyperparameter Tuning

In [ ]:
tuned_models = {}

for model_cfg in MODELS:
    name = model_cfg['name']
    
    print(f"\nTuning {name} for {TUNE_ITERATIONS} iterations")
    
    model = RTDETR(model_cfg['weights']) if 'rtdetr' in name.lower() else YOLO(model_cfg['weights'])
    tuned_models[name] = model.tune(
        data=str(YOLO_DIR / "config.yaml"),
        epochs=50,
        iterations=TUNE_ITERATIONS,
        imgsz=IMG_SIZE,
        plots=False,
        save=True,
        val=True
    )
    
    print(f"{name} tuning complete. Best model saved.")

print(f"\nAll tuning complete.")


Tuning yolov8l for 10 iterations
Tuner: Initialized Tuner instance with 'tune_dir=/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/tune28'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0, 'close_mosaic': 10}
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 

## Train Models

In [ ]:
results = []

for idx, model_cfg in enumerate(MODELS, 1):
    name = model_cfg['name']
    print(f"\n[{idx}/{len(MODELS)}] Training {name}")
    
    if 'tuned_models' in globals() and name in tuned_models:
        model = tuned_models[name]
        print(f"Using tuned model")
    else:
        model = RTDETR(model_cfg['weights']) if 'rtdetr' in name.lower() else YOLO(model_cfg['weights'])
        print(f"Using default hyperparameters")
    
    model.train(
        data=str(YOLO_DIR / "config.yaml"),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        patience=PATIENCE,
        batch=BATCH,
        seed=SEED,
        name=f"{exp_id}_{name}",
        project="runs",
        plots=False,
        verbose=False
    )
    
    val_metrics = model.val(split='val', verbose=False)
    test_metrics = model.val(split='test', verbose=False)
    
    val_p, val_r = float(val_metrics.box.mp), float(val_metrics.box.mr)
    test_p, test_r = float(test_metrics.box.mp), float(test_metrics.box.mr)
    
    val_f1 = 2 * (val_p * val_r) / (val_p + val_r + 1e-6)
    test_f1 = 2 * (test_p * test_r) / (test_p + test_r + 1e-6)
    
    results.append({
        'model': name,
        'tuned': 'tuned_models' in globals() and name in tuned_models,
        'val_precision': val_p,
        'val_recall': val_r,
        'val_f1': val_f1,
        'val_map50': float(val_metrics.box.map50),
        'test_precision': test_p,
        'test_recall': test_r,
        'test_f1': test_f1,
        'test_map50': float(test_metrics.box.map50),
    })
    
    print(f"{name}: val_f1={val_f1:.4f}, test_f1={test_f1:.4f}, test_map50={results[-1]['test_map50']:.4f}")

## Results

In [ ]:
df = pd.DataFrame(results)

summary = {
    'exp_id': exp_id,
    'seed': SEED,
    'epochs': EPOCHS,
    'img_size': IMG_SIZE,
    'batch': BATCH,
    'patience': PATIENCE,
    'models': [m['name'] for m in MODELS],
    'results': results,
    'best_model': results[df['test_map50'].idxmax()]['model'] if len(results) > 0 else None,
    'best_test_map50': float(df['test_map50'].max()) if len(results) > 0 else 0.0
}

with open(RESULTS_DIR / f"{exp_id}.json", 'w') as f:
    json.dump(summary, f, indent=2)

print("\nResults")
print(df[['model', 'val_f1', 'val_map50', 'test_f1', 'test_map50']].to_string(index=False))
print(f"\nBest: {summary['best_model']} (test_map50={summary['best_test_map50']:.4f})")
print(f"Saved: results/{exp_id}.json")

## Analysis

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for model_cfg in MODELS:
    name = model_cfg['name']
    csv_file = Path(f"runs/{exp_id}_{name}/results.csv")
    
    if csv_file.exists():
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.strip()
        
        p = df['metrics/precision(B)']
        r = df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        
        axes[0].plot(f1, label=name, linewidth=2)
        axes[1].plot(df['metrics/mAP50(B)'], label=name, linewidth=2)

axes[0].set_title('F1 Score')
axes[0].set_ylabel('F1')
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].set_title('mAP50')
axes[1].set_ylabel('mAP50')
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / f"curves_{exp_id}.png", dpi=150)
plt.show()